In [2]:
"""
Excel数据清洗脚本 - Jupyter Notebook版本
功能:
1. 检查index和raw_index是否一致
2. 过滤species为空的行
3. 验证并提取locations中的struc_addr
4. 处理times字段,为空时从date列提取,异常标记供审核
"""

import pandas as pd
import ast
from pathlib import Path

def clean_data(input_file, output_file=None):
    """
    清洗Excel数据
    
    Args:
        input_file: 输入xlsx文件路径
        output_file: 输出xlsx文件路径(可选,默认为input_cleaned.xlsx)
    
    Returns:
        df_cleaned: 清洗后的DataFrame
    """
    
    # 读取Excel文件
    print(f"读取文件: {input_file}")
    df = pd.read_excel(input_file)
    
    # 验证列名
    required_cols = ['index', 'raw_index', 'species', 'locations', 'times', 'date']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"缺少必需的列: {missing_cols}")
    
    print(f"\n原始数据行数: {len(df)}")
    
    # 1. 检查index和raw_index是否一致
    print("\n=== 检查index和raw_index一致性 ===")
    mismatch = df[df['index'] != df['raw_index']]
    if len(mismatch) > 0:
        print(f"⚠️  发现 {len(mismatch)} 条不一致的记录:")
        for idx, row in mismatch.head(10).iterrows():
            print(f"  行 {idx+2}: index={row['index']}, raw_index={row['raw_index']}")
        if len(mismatch) > 10:
            print(f"  ... 还有 {len(mismatch)-10} 条")
    else:
        print("✓ 所有index和raw_index一致")
    
    # 2. 删除species为空的行
    print("\n=== 过滤species为空的行 ===")
    df_cleaned = df.copy()
    empty_species = df_cleaned['species'].isna() | (df_cleaned['species'] == '')
    empty_count = empty_species.sum()
    
    if empty_count > 0:
        print(f"删除 {empty_count} 条species为空的记录")
        df_cleaned = df_cleaned[~empty_species].copy()
    else:
        print("✓ 没有species为空的记录")
    
    print(f"过滤后数据行数: {len(df_cleaned)}")
    
    # 3. 处理locations字段
    print("\n=== 处理locations字段 ===")
    
    def extract_struc_addr(loc_str):
        """
        从locations字符串中提取struc_addr
        支持两种格式:
        1. dict: {'extra_addr': '...', 'struc_addr': '...'}
        2. list(dict): [{'extra_addr': '...', 'struc_addr': '...'}, ...]
        """
        if pd.isna(loc_str) or loc_str == '':
            return None
        
        try:
            # 尝试将字符串转换为Python对象
            loc_obj = ast.literal_eval(str(loc_str))
            
            # 如果是列表,取第一个元素
            if isinstance(loc_obj, list) and len(loc_obj) > 0:
                loc_obj = loc_obj[0]
            
            # 检查是否为字典且包含struc_addr
            if isinstance(loc_obj, dict) and 'struc_addr' in loc_obj:
                return loc_obj['struc_addr']
            else:
                return None
        except:
            return None
    
    # 提取struc_addr
    df_cleaned['struc_addr'] = df_cleaned['locations'].apply(extract_struc_addr)
    
    # 统计异常的locations
    valid_locations = df_cleaned['struc_addr'].notna()
    invalid_count = (~valid_locations).sum()
    
    if invalid_count > 0:
        print(f"⚠️  发现 {invalid_count} 条locations格式异常的记录,将删除:")
        invalid_samples = df_cleaned[~valid_locations].head(5)
        for idx, row in invalid_samples.iterrows():
            loc_preview = str(row['locations'])[:100] + '...' if len(str(row['locations'])) > 100 else str(row['locations'])
            print(f"  行 {idx+2}: {loc_preview}")
        if invalid_count > 5:
            print(f"  ... 还有 {invalid_count-5} 条")
        df_cleaned = df_cleaned[valid_locations].copy()
    else:
        print("✓ 所有locations格式正确")
    
    print(f"处理后数据行数: {len(df_cleaned)}")
    
    # 4. 处理times字段
    print("\n=== 处理times字段 ===")
    
    def extract_date_from_date_col(date_str):
        """从date列提取时间,转换为yyyy-mm格式"""
        if pd.isna(date_str) or date_str == '':
            return None
        
        try:
            date_str = str(date_str).strip()
            
            # 尝试解析日期
            import re
            # 匹配yyyy-mm-dd或yyyy/mm/dd或yyyy.mm.dd格式
            pattern = r'(\d{4})[-/.](\d{1,2})'
            match = re.search(pattern, date_str)
            
            if match:
                year, month = int(match.group(1)), int(match.group(2))
                if 1900 <= year <= 2100 and 1 <= month <= 12:
                    return f"{year:04d}-{month:02d}"
            
            return None
        except:
            return None
    
    def process_time(time_str, date_str):
        """
        处理时间字段,返回格式为yyyy-mm
        优先使用times,异常时从date列提取
        """
        # 尝试处理times
        if pd.notna(time_str) and time_str != '':
            try:
                time_str = str(time_str).strip()
                
                # 如果是列表,只保留第一个
                if time_str.startswith('['):
                    time_list = ast.literal_eval(time_str)
                    if isinstance(time_list, list) and len(time_list) > 0:
                        time_str = str(time_list[0])
                    else:
                        # 列表为空,尝试从date提取
                        extracted = extract_date_from_date_col(date_str)
                        return extracted if extracted else None, 'from_date'
                
                # 验证格式 yyyy-mm
                if len(time_str) >= 7:
                    parts = time_str[:7].split('-')
                    if len(parts) == 2:
                        year, month = int(parts[0]), int(parts[1])
                        if 1900 <= year <= 2100 and 1 <= month <= 12:
                            return f"{year:04d}-{month:02d}", 'ok'
            except:
                pass
        
        # times为空或异常,尝试从date提取
        extracted = extract_date_from_date_col(date_str)
        if extracted:
            return extracted, 'from_date'
        else:
            return None, 'failed'
    
    # 处理times
    results = df_cleaned.apply(lambda row: process_time(row['times'], row['date']), axis=1)
    df_cleaned['times_clean'] = results.apply(lambda x: x[0])
    df_cleaned['time_source'] = results.apply(lambda x: x[1])
    
    # 统计状态
    status_counts = df_cleaned['time_source'].value_counts()
    print("\n时间处理状态统计:")
    for status, count in status_counts.items():
        status_desc = {
            'ok': '✓ 原始times格式正确',
            'from_date': '⚠️  从date列提取',
            'failed': '✗ times和date都无法提取'
        }
        print(f"  {status_desc.get(status, status)}: {count} 条")
    
    # 删除无法提取时间的记录
    failed_mask = df_cleaned['time_source'] == 'failed'
    failed_count = failed_mask.sum()
    
    if failed_count > 0:
        print(f"\n删除 {failed_count} 条无法提取时间的记录:")
        failed_samples = df_cleaned[failed_mask].head(5)
        for idx, row in failed_samples.iterrows():
            print(f"  行 {idx+2}: times={row['times']}, date={row['date']}")
        if failed_count > 5:
            print(f"  ... 还有 {failed_count-5} 条")
        df_cleaned = df_cleaned[~failed_mask].copy()
    
    print(f"\n最终数据行数: {len(df_cleaned)}")
    
    # 5. 保存清洗后的数据
    if output_file is None:
        input_path = Path(input_file)
        output_file = input_path.parent / f"{input_path.stem}_cleaned.xlsx"
    
    print(f"\n保存清洗后的数据到: {output_file}")
    df_cleaned.to_excel(output_file, index=False)
    
    # 生成清洗报告
    print("\n" + "="*60)
    print("数据清洗摘要")
    print("="*60)
    print(f"原始记录数: {len(df)}")
    print(f"清洗后记录数: {len(df_cleaned)}")
    print(f"删除记录数: {len(df) - len(df_cleaned)}")
    print(f"  - species为空: {empty_count}")
    print(f"  - locations异常: {invalid_count}")
    print(f"  - times和date都无法提取: {failed_count}")
    print(f"\n新增列(保留所有原始列):")
    print(f"  - struc_addr: 从locations提取的结构化地址")
    print(f"  - times_clean: 清洗后的时间(yyyy-mm格式)")
    print(f"  - time_source: 时间来源标记 (ok/from_date)")
    print("="*60)
    
    return df_cleaned

df_cleaned = clean_data('uie结果完整版.xlsx', 'output/uie清洗待整理&人工审核.xlsx')

读取文件: uie结果完整版.xlsx

原始数据行数: 5900

=== 检查index和raw_index一致性 ===
✓ 所有index和raw_index一致

=== 过滤species为空的行 ===
删除 748 条species为空的记录
过滤后数据行数: 5152

=== 处理locations字段 ===
⚠️  发现 1 条locations格式异常的记录,将删除:
  行 4519: {'extra_addr': None, 'struc_addr': None}
处理后数据行数: 5151

=== 处理times字段 ===

时间处理状态统计:
  ✓ 原始times格式正确: 4834 条
  ⚠️  从date列提取: 317 条

最终数据行数: 5151

保存清洗后的数据到: output/uie清洗待整理&人工审核.xlsx

数据清洗摘要
原始记录数: 5900
清洗后记录数: 5151
删除记录数: 749
  - species为空: 748
  - locations异常: 1
  - times和date都无法提取: 0

新增列(保留所有原始列):
  - struc_addr: 从locations提取的结构化地址
  - times_clean: 清洗后的时间(yyyy-mm格式)
  - time_source: 时间来源标记 (ok/from_date)


In [ ]:
"""
地址处理和验证脚本 - Jupyter Notebook版本
功能:
1. 过滤无效地址(包含"中国"、"全国"、"未指定"等关键词)
2. 过滤国外地址
3. 处理多地址(取第一个)
4. 标准化地址格式(提取前3段为省-市-区)
5. 保留不完整地址(省或省-市)
"""

import pandas as pd
import re
from pathlib import Path


def validate_struc_addr(input_file, output_file=None):
    """
    验证struc_addr字段的有效性
    
    Args:
        input_file: 输入xlsx文件路径
        output_file: 输出xlsx文件路径(可选,默认为input_validated.xlsx)
    
    Returns:
        df_validated: 验证后的DataFrame
    """
    
    # 读取Excel文件
    print(f"读取文件: {input_file}")
    df = pd.read_excel(input_file)
    
    # 验证列名
    if 'struc_addr' not in df.columns:
        raise ValueError("文件中缺少 'struc_addr' 列")
    
    print(f"\n原始数据行数: {len(df)}")
    
    df_validated = df.copy()
    
    # 1. 过滤无效地址关键词
    print("\n=== 过滤无效地址 ===")
    
    # 定义需要过滤的关键词
    invalid_keywords = [
        '中国', '全国范围', '全国', '未指定', '未提及', '未明确提及', 
        '未明确', '未知', '不详', '待定', 'unknown', 'Unknown'
    ]
    
    def contains_invalid_keyword(addr_str):
        """检查地址是否包含无效关键词"""
        if pd.isna(addr_str) or addr_str == '':
            return False
        
        addr_str = str(addr_str)
        for keyword in invalid_keywords:
            if keyword in addr_str:
                return True
        return False
    
    invalid_keyword_mask = df_validated['struc_addr'].apply(contains_invalid_keyword)
    invalid_keyword_count = invalid_keyword_mask.sum()
    
    if invalid_keyword_count > 0:
        print(f"删除包含无效关键词的地址: {invalid_keyword_count} 条")
        invalid_samples = df_validated[invalid_keyword_mask].head(10)
        for idx, row in invalid_samples.iterrows():
            print(f"  行 {idx+2}: {row['struc_addr']}")
        if invalid_keyword_count > 10:
            print(f"  ... 还有 {invalid_keyword_count-10} 条")
        df_validated = df_validated[~invalid_keyword_mask].copy()
    else:
        print("✓ 没有包含无效关键词的地址")
    
    print(f"过滤后数据行数: {len(df_validated)}")
    
    # 2. 过滤国外地址
    print("\n=== 过滤国外地址 ===")
    
    # 中国省级行政区列表(包括省、自治区、直辖市、特别行政区)
    china_provinces = [
        '北京市', '天津市', '上海市', '重庆市',  # 直辖市
        '河北省', '山西省', '辽宁省', '吉林省', '黑龙江省',  # 省
        '江苏省', '浙江省', '安徽省', '福建省', '江西省', '山东省',
        '河南省', '湖北省', '湖南省', '广东省', '海南省',
        '四川省', '贵州省', '云南省', '陕西省', '甘肃省', '青海省', '台湾省',
        '内蒙古自治区', '广西壮族自治区', '西藏自治区', '宁夏回族自治区', '新疆维吾尔自治区',  # 自治区
        '香港特别行政区', '澳门特别行政区',  # 特别行政区
        # 简称形式
        '北京', '天津', '上海', '重庆',
        '河北', '山西', '辽宁', '吉林', '黑龙江',
        '江苏', '浙江', '安徽', '福建', '江西', '山东',
        '河南', '湖北', '湖南', '广东', '海南',
        '四川', '贵州', '云南', '陕西', '甘肃', '青海', '台湾',
        '内蒙古', '广西', '西藏', '宁夏', '新疆',
        '香港', '澳门'
    ]
    
    def is_china_address(addr_str):
        """检查是否为中国地址"""
        if pd.isna(addr_str) or addr_str == '':
            return False
        
        addr_str = str(addr_str)
        
        # 检查第一段是否为中国省份
        first_part = addr_str.split('-')[0] if '-' in addr_str else addr_str
        
        return first_part in china_provinces
    
    china_mask = df_validated['struc_addr'].apply(is_china_address)
    foreign_count = (~china_mask).sum()
    
    if foreign_count > 0:
        print(f"删除国外地址: {foreign_count} 条")
        foreign_samples = df_validated[~china_mask].head(10)
        for idx, row in foreign_samples.iterrows():
            print(f"  行 {idx+2}: {row['struc_addr']}")
        if foreign_count > 10:
            print(f"  ... 还有 {foreign_count-10} 条")
        df_validated = df_validated[china_mask].copy()
    else:
        print("✓ 没有国外地址")
    
    print(f"过滤后数据行数: {len(df_validated)}")
    
    # 3. 处理和验证地址有效性
    print("\n=== 处理struc_addr字段 ===")
    
    def process_address(addr_str):
        """
        处理地址:
        1. 如果包含多个地址(逗号或分号分隔),取第一个
        2. 提取标准的xx-xx-xx格式(前3段)
        返回: (processed_addr, status, note)
        """
        if pd.isna(addr_str) or addr_str == '':
            return None, 'empty', '地址为空'
        
        addr_str = str(addr_str).strip()
        
        # 处理多地址情况(用逗号、分号、顿号分隔)
        if ',' in addr_str or '，' in addr_str or ';' in addr_str or '；' in addr_str or '、' in addr_str:
            # 分割并取第一个
            for sep in [',', '，', ';', '；', '、']:
                if sep in addr_str:
                    parts = addr_str.split(sep)
                    addr_str = parts[0].strip()
                    break
        
        # 检查是否以-开头或结尾
        if addr_str.startswith('-') or addr_str.endswith('-'):
            addr_str = addr_str.strip('-')
        
        # 分割地址
        parts = addr_str.split('-')
        
        # 移除空段
        parts = [part.strip() for part in parts if part.strip()]
        
        if len(parts) == 0:
            return None, 'empty', '地址为空'
        elif len(parts) == 1:
            # 只有一段,保留原样(可能是省级)
            return parts[0], 'incomplete_kept', '仅有省级(已保留)'
        elif len(parts) == 2:
            # 两段,保留原样(省-市)
            return '-'.join(parts), 'incomplete_kept', '省-市(已保留)'
        elif len(parts) == 3:
            # 标准格式
            return '-'.join(parts), 'valid', '标准格式'
        else:
            # 超过3段,只取前3段
            return '-'.join(parts[:3]), 'trimmed', f'多段地址(从{len(parts)}段截取前3段)'
    
    # 处理地址
    process_results = df_validated['struc_addr'].apply(process_address)
    df_validated['struc_addr_clean'] = process_results.apply(lambda x: x[0])
    df_validated['addr_status'] = process_results.apply(lambda x: x[1])
    df_validated['addr_note'] = process_results.apply(lambda x: x[2])
    
    # 删除处理后仍为空的地址
    empty_mask = df_validated['struc_addr_clean'].isna()
    empty_count = empty_mask.sum()
    
    if empty_count > 0:
        print(f"删除处理后仍为空的地址: {empty_count} 条")
        df_validated = df_validated[~empty_mask].copy()
    
    # 统计结果
    status_counts = df_validated['addr_status'].value_counts()
    
    print("\n地址处理统计:")
    print(f"{'状态':<20} {'数量':<10} {'说明'}")
    print("-" * 60)
    
    status_desc = {
        'valid': '✓ 标准格式(省-市-区)',
        'incomplete_kept': '⚠️  不完整但已保留(省或省-市)',
        'trimmed': '⚠️  多地址截取前3段',
        'empty': '✗ 地址为空'
    }
    
    for status in ['valid', 'incomplete_kept', 'trimmed', 'empty']:
        if status in status_counts:
            count = status_counts[status]
            desc = status_desc.get(status, status)
            percentage = count / len(df_validated) * 100
            print(f"{desc:<20} {count:<10} ({percentage:.1f}%)")
    
    # 显示处理样例
    processed_mask = df_validated['addr_status'].isin(['incomplete_kept', 'trimmed'])
    processed_count = processed_mask.sum()
    
    if processed_count > 0:
        print(f"\n处理的地址样例(前10条):")
        processed_samples = df_validated[processed_mask].head(10)
        
        for idx, row in processed_samples.iterrows():
            print(f"\n  行 {idx+2}:")
            print(f"    原地址: {row['struc_addr']}")
            print(f"    处理后: {row['struc_addr_clean']}")
            print(f"    说明: {row['addr_note']}")
        
        if processed_count > 10:
            print(f"\n  ... 还有 {processed_count-10} 条处理的地址")
    
    print(f"\n最终数据行数: {len(df_validated)}")
    
    # 保存验证后的数据
    if output_file is None:
        input_path = Path(input_file)
        output_file = input_path.parent / f"{input_path.stem}_validated.xlsx"
    
    print(f"\n\n保存验证后的数据到: {output_file}")
    df_validated.to_excel(output_file, index=False)
    
    # 生成验证报告
    print("\n" + "="*60)
    print("地址处理摘要")
    print("="*60)
    print(f"原始记录数: {len(df)}")
    print(f"处理后记录数: {len(df_validated)}")
    print(f"删除记录数: {len(df) - len(df_validated)}")
    print(f"  - 包含无效关键词: {invalid_keyword_count} 条")
    print(f"  - 国外地址: {foreign_count} 条")
    print(f"  - 地址为空: {empty_count} 条")
    print(f"\n地址格式统计:")
    print(f"  标准格式(省-市-区): {status_counts.get('valid', 0)} 条")
    print(f"  不完整但保留(省/省-市): {status_counts.get('incomplete_kept', 0)} 条")
    print(f"  多地址截取: {status_counts.get('trimmed', 0)} 条")
    print(f"\n新增列:")
    print(f"  - struc_addr_clean: 处理后的标准地址")
    print(f"  - addr_status: 地址状态 (valid/incomplete_kept/trimmed)")
    print(f"  - addr_note: 处理说明")
    print("="*60)
    
    return df_validated

df_validated = validate_struc_addr('output/uie清洗待整理&人工审核.xlsx')

读取文件: output/uie清洗待整理&人工审核.xlsx

原始数据行数: 5151

=== 过滤无效地址 ===
删除包含无效关键词的地址: 396 条
  行 31: 全国范围
  行 57: 中国-南方地区-未指定
  行 148: 中国
  行 155: 中国-全国-全国
  行 157: 浙江省-未知市-未知区, 江西省-未知市-未知区
  行 158: ['黑龙江省-未知-未知', '吉林省-未知-未知', '辽宁省-未知-未知', '湖北省-未知-未知', '江苏省-未知-未知', '山东省-未知-未知', '浙江省-未知-未知', '江西省-未知-未知', '安徽省-未知-未知']
  行 165: 吉林省-长春市-未明确区
  行 176: 中国
  行 180: 辽宁省-沈阳市-未知区
  行 187: 中国
  ... 还有 386 条
过滤后数据行数: 4755

=== 过滤国外地址 ===
删除国外地址: 113 条
  行 25: 华南地区
  行 47: 西南地区
  行 67: 墨西哥-东南部
  行 126: 东北地区
  行 184: 俄罗斯-莫斯科-莫斯科
  行 248: 沿海各省-沿海各市-沿海各区
  行 254: 沿海各省-沿海各市-沿海各区
  行 268: 美国-大西洋沿岸
  行 291: 环渤海地区
  行 414: 沿海各省市-沿海各市-沿海各区
  ... 还有 103 条
过滤后数据行数: 4642

=== 处理struc_addr字段 ===

地址处理统计:
状态                   数量         说明
------------------------------------------------------------
✓ 标准格式(省-市-区)        4282       (92.2%)
⚠️  不完整但已保留(省或省-市)   358        (7.7%)
⚠️  多地址截取前3段         2          (0.0%)

处理的地址样例(前10条):

  行 61:
    原地址: 海南省-儋州市
    处理后: 海南省-儋州市
    说明: 省-市(已保留)

  行 70:
    原地址: 广西壮族自治区-百色市
  

In [10]:
"""
物种处理和入侵物种识别脚本 - Jupyter Notebook版本
功能:
1. 解析species字段中的多物种(用逗号分隔)
2. 根据入侵物种名录匹配入侵物种
3. 删除不包含入侵物种的记录
4. 统计原始缺失率和入侵物种频次
"""

import pandas as pd
import re
from pathlib import Path


def process_species(input_file, output_file=None):
    """
    处理species字段并识别入侵物种
    
    Args:
        input_file: 输入xlsx文件路径
        output_file: 输出xlsx文件路径(可选,默认为input_species_processed.xlsx)
    
    Returns:
        df_processed: 处理后的DataFrame
    """
    
    # 入侵物种名录(根据附表1)
    alien_species_dict = {
        '紫茎泽兰': 'Ageratina adenophora',
        '藿香蓟': 'Ageratum conyzoides',
        '胜红蓟': 'Ageratum conyzoides',  # 藿香蓟的别名
        '空心莲子草': 'Alternanthera philoxeroides',
        '长芒苋': 'Amaranthus palmeri',
        '刺苋': 'Amaranthus spinosus',
        '豚草': 'Ambrosia artemisiifolia',
        '三裂叶豚草': 'Ambrosia trifida',
        '落葵薯': 'Anredera cordifolia',
        '野燕麦': 'Avena fatua',
        '三叶鬼针草': 'Bidens pilosa',
        '鬼针草': 'Bidens pilosa',  # 三叶鬼针草的简称
        '水盾草': 'Cabomba caroliniana',
        '长刺蒺藜草': 'Cenchrus longispinus',
        '飞机草': 'Chromolaena odorata',
        '凤眼蓝': 'Eichhornia crassipes',
        '水葫芦': 'Eichhornia crassipes',  # 凤眼蓝的别名
        '小蓬草': 'Erigeron canadensis',
        '苏门白酒草': 'Erigeron sumatrensis',
        '黄顶菊': 'Flaveria bidentis',
        '五爪金龙': 'Ipomoea cairica',
        '假苍耳': 'Cyclachaena xanthiifolia',
        '马缨丹': 'Lantana camara',
        '毒莴苣': 'Lactuca serriola',
        '薇甘菊': 'Mikania micrantha',
        '小花蔓泽兰': 'Mikania micrantha',  # 薇甘菊的别名
        '光荚含羞草': 'Mimosa bimucronata',
        '银胶菊': 'Parthenium hysterophorus',
        '垂序商陆': 'Phytolacca americana',
        '美洲商陆': 'Phytolacca americana',  # 垂序商陆的别名
        '大薸': 'Pistia stratiotes',
        '假臭草': 'Praxelis clematidea',
        '刺果瓜': 'Sicyos angulatus',
        '黄花刺茄': 'Solanum rostratum',
        '加拿大一枝黄花': 'Solidago canadensis',
        '一枝黄花': 'Solidago canadensis',  # 加拿大一枝黄花的简称
        '假高粱': 'Sorghum halepense',
        '互花米草': 'Spartina alterniflora',
        '刺苍耳': 'Xanthium spinosum',
    }
    
    # 读取Excel文件
    print(f"读取文件: {input_file}")
    df = pd.read_excel(input_file)
    
    # 验证列名
    if 'species' not in df.columns:
        raise ValueError("文件中缺少 'species' 列")
    
    print(f"\n原始数据行数: {len(df)}")
    
    df_processed = df.copy()
    
    # 处理species字段
    print("\n=== 处理species字段并识别入侵物种 ===")
    
    def extract_alien_species(species_str):
        """
        从species字符串中提取入侵物种
        返回: (alien_species_list, all_species_list)
        """
        if pd.isna(species_str) or species_str == '':
            return [], []
        
        species_str = str(species_str).strip()
        
        # 分割物种(用逗号、顿号、分号等分隔)
        separators = [',', '，', '、', ';', '；']
        for sep in separators:
            if sep in species_str:
                species_list = [s.strip() for s in species_str.split(sep) if s.strip()]
                break
        else:
            # 没有分隔符,单个物种
            species_list = [species_str]
        
        # 匹配入侵物种
        alien_species = []
        for species in species_list:
            if species in alien_species_dict:
                if alien_species_dict[species] is not None:
                    alien_species.append(species)
        
        return alien_species, species_list
    
    # 提取入侵物种
    extract_results = df_processed['species'].apply(extract_alien_species)
    df_processed['alien_species'] = extract_results.apply(
        lambda x: ', '.join(x[0]) if x[0] else None
    )
    df_processed['all_species_parsed'] = extract_results.apply(
        lambda x: ', '.join(x[1]) if x[1] else None
    )
    
    # 统计结果
    has_alien = df_processed['alien_species'].notna()
    alien_count = has_alien.sum()
    no_alien_count = (~has_alien).sum()
    
    total_count = len(df_processed)
    missing_rate = (no_alien_count / total_count * 100) if total_count > 0 else 0
    
    print(f"\n入侵物种识别统计:")
    print(f"{'类别':<25} {'数量':<10} {'比例'}")
    print("-" * 60)
    print(f"{'包含入侵物种':<25} {alien_count:<10} ({alien_count/total_count*100:.1f}%)")
    print(f"{'不包含入侵物种':<25} {no_alien_count:<10} ({missing_rate:.1f}%)")
    print(f"{'总计':<25} {total_count:<10} (100.0%)")
    
    print(f"\n原始缺失率: {missing_rate:.2f}%")
    
    # 删除不包含入侵物种的记录
    if no_alien_count > 0:
        print(f"\n删除不包含入侵物种的记录: {no_alien_count} 条")
        no_alien_samples = df_processed[~has_alien].head(10)
        for idx, row in no_alien_samples.iterrows():
            print(f"  行 {idx+2}: {row['species']}")
        if no_alien_count > 10:
            print(f"  ... 还有 {no_alien_count-10} 条")
        
        df_processed = df_processed[has_alien].copy()
        print(f"\n删除后数据行数: {len(df_processed)}")
    else:
        print("\n✓ 所有记录都包含入侵物种")
    
    # 显示识别到的入侵物种样例
    if len(df_processed) > 0:
        print(f"\n识别到入侵物种的样例(前10条):")
        alien_samples = df_processed.head(10)
        
        for idx, row in alien_samples.iterrows():
            print(f"\n  行 {idx+2}:")
            print(f"    原始species: {row['species']}")
            print(f"    入侵物种: {row['alien_species']}")
        
        if len(df_processed) > 10:
            print(f"\n  ... 还有 {len(df_processed)-10} 条")
    
    # 统计各入侵物种出现次数
    if len(df_processed) > 0:
        print("\n\n入侵物种出现频次统计(Top 15):")
        all_aliens = []
        for aliens_str in df_processed['alien_species'].dropna():
            all_aliens.extend([s.strip() for s in aliens_str.split(',')])
        
        alien_freq = pd.Series(all_aliens).value_counts()
        print(f"\n{'物种名称':<20} {'出现次数':<10}")
        print("-" * 40)
        for species, count in alien_freq.head(15).items():
            print(f"{species:<20} {count:<10}")
    
    # 保存处理后的数据
    if output_file is None:
        input_path = Path(input_file)
        output_file = input_path.parent / f"{input_path.stem}_species_processed.xlsx"
    
    print(f"\n\n保存处理后的数据到: {output_file}")
    df_processed.to_excel(output_file, index=False)
    
    # 生成处理报告
    print("\n" + "="*60)
    print("物种处理摘要")
    print("="*60)
    print(f"原始记录数: {total_count}")
    print(f"处理后记录数: {len(df_processed)}")
    print(f"删除记录数: {no_alien_count}")
    print(f"  - 不包含入侵物种: {no_alien_count} 条")
    print(f"\n原始缺失率: {missing_rate:.2f}%")
    print(f"最终保留: 100% 包含入侵物种")
    print(f"\n新增列:")
    print(f"  - alien_species: 识别到的入侵物种(逗号分隔)")
    print(f"  - all_species_parsed: 解析后的所有物种列表")
    print(f"\n入侵物种名录共包含: {len([k for k, v in alien_species_dict.items() if v is not None])} 个物种")
    print("="*60)
    
    return df_processed

df_processed = process_species('output/uie清洗待整理&人工审核_validated.xlsx')

读取文件: output/uie清洗待整理&人工审核_validated.xlsx

原始数据行数: 4642

=== 处理species字段并识别入侵物种 ===

入侵物种识别统计:
类别                        数量         比例
------------------------------------------------------------
包含入侵物种                    4334       (93.4%)
不包含入侵物种                   308        (6.6%)
总计                        4642       (100.0%)

原始缺失率: 6.64%

删除不包含入侵物种的记录: 308 条
  行 24: 小花宽叶十万错
  行 25: 扶桑绵粉蚧
  行 29: 三裂叶蟛蜞菊
  行 31: 三裂叶蟛蜞菊
  行 57: 夏孢锈菌, 粉孢属, 泽兰尾孢, 刺盘孢, 交链孢, 长蠕孢霉, 弯孢霉, 夏孢锈菌, 黑孢霉
  行 59: Ramularia caricis, Ramularia concomitans
  行 106: 三裂叶豚草锈菌
  行 166: 夜蛾, 象甲, 叶甲, 瘿蝇, 长角象甲, 豚草条纹叶甲, 白锈菌, 万寿菊叶斑病菌, 豚草木质部抑制细菌, 三裂叶豚草锈菌, 多年生禾本科牧草, 紫穗槐
  行 173: 苍耳属非中国种Xanthium spp.non-Chinese, 三裂叶豚草Ambrosia trifi da Linn., 墨天牛非中国种Monochamus spp.non-Chinese
  行 196: 豚草属Ambrosia
  ... 还有 298 条

删除后数据行数: 4334

识别到入侵物种的样例(前10条):

  行 2:
    原始species: 三叶鬼针草, 马唐
    入侵物种: 三叶鬼针草

  行 3:
    原始species: 三叶鬼针草
    入侵物种: 三叶鬼针草

  行 4:
    原始species: 三叶鬼针草
    入侵物种: 三叶鬼针草

  行 5:
    原始species: 三叶鬼针草
    入侵物种: 三叶鬼针草

  行 

In [14]:
"""
入侵物种时空分布可视化脚本 - 改进版
功能:
1. 使用真实的lng, lat经纬度数据
2. 添加中国地图作为底图
3. 绘制不同物种的空间分布图(不同颜色)
4. 绘制时间演变动画/多图
"""

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle, Polygon
import numpy as np
from pathlib import Path
import seaborn as sns
import json
import requests
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号


def load_china_map():
    """
    加载中国地图底图
    """
    china_url = "https://geo.datav.aliyun.com/areas_v3/bound/100000.json"
    try:
        print("正在加载中国地图底图...")
        response = requests.get(china_url, timeout=10)
        china_geojson = response.json()
        print("✓ 地图加载成功")
        return china_geojson
    except Exception as e:
        print(f"警告：无法加载在线地图 ({e})，将仅绘制坐标轴。")
        return None


def plot_china_basemap(ax, china_geojson):
    """
    在指定axes上绘制中国地图底图
    """
    if china_geojson is None:
        return
    
    try:
        features = china_geojson.get('features', [])
        for feature in features:
            geometry = feature.get('geometry', {})
            geo_type = geometry.get('type', '')
            coordinates = geometry.get('coordinates', [])
            
            if geo_type == 'Polygon':
                for poly_coords in coordinates:
                    poly = Polygon(poly_coords, facecolor='#ebebeb', 
                                 edgecolor='#999999', alpha=0.5, linewidth=0.5)
                    ax.add_patch(poly)
            elif geo_type == 'MultiPolygon':
                for multi_poly in coordinates:
                    for poly_coords in multi_poly:
                        poly = Polygon(poly_coords, facecolor='#ebebeb', 
                                     edgecolor='#999999', alpha=0.5, linewidth=0.5)
                        ax.add_patch(poly)
    except Exception as e:
        print(f"绘制地图时出错: {e}")


def visualize_species_distribution(input_file, output_dir=None):
    """
    可视化入侵物种的时空分布
    
    Args:
        input_file: 输入xlsx文件路径
        output_dir: 输出图片目录(可选)
    """
    
    # 读取数据
    print(f"读取文件: {input_file}")
    df = pd.read_excel(input_file)
    
    # 验证列名
    required_cols = ['alien_species', 'times_clean', 'lng', 'lat']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"文件中缺少必需的列: {missing_cols}")
    
    print(f"\n数据行数: {len(df)}")
    
    # 使用真实经纬度
    df['longitude'] = pd.to_numeric(df['lng'], errors='coerce')
    df['latitude'] = pd.to_numeric(df['lat'], errors='coerce')
    
    # 移除坐标缺失的记录
    valid_coords = df['longitude'].notna() & df['latitude'].notna()
    df_plot = df[valid_coords].copy()
    print(f"有效坐标数: {len(df_plot)}")
    
    # 解析时间
    df_plot['year'] = df_plot['times_clean'].apply(lambda x: str(x)[:4] if pd.notna(x) else None)
    df_plot['year_month'] = df_plot['times_clean']
    
    # 统计物种
    all_species = []
    for species_str in df_plot['alien_species'].dropna():
        all_species.extend([s.strip() for s in str(species_str).split(',')])
    
    species_counts = pd.Series(all_species).value_counts()
    print(f"\n物种数量: {len(species_counts)}")
    print(f"前10种物种:")
    for sp, cnt in species_counts.head(10).items():
        print(f"  {sp}: {cnt}")
    
    # 选择Top物种进行可视化
    top_n = min(15, len(species_counts))
    top_species = species_counts.head(top_n).index.tolist()
    
    # 为每个物种分配颜色
    colors = plt.cm.tab20(np.linspace(0, 1, len(top_species)))
    species_colors = dict(zip(top_species, colors))
    
    # 为每条记录分配主要物种(取第一个入侵物种)
    def get_primary_species(species_str):
        if pd.isna(species_str):
            return 'Other'
        species_list = [s.strip() for s in str(species_str).split(',')]
        for sp in species_list:
            if sp in top_species:
                return sp
        return 'Other'
    
    df_plot['primary_species'] = df_plot['alien_species'].apply(get_primary_species)
    
    # 设置输出目录
    if output_dir is None:
        output_dir = Path(input_file).parent / 'visualizations'
    else:
        output_dir = Path(output_dir)
    
    output_dir.mkdir(exist_ok=True, parents=True)
    print(f"\n图片将保存到: {output_dir}")
    
    # 加载中国地图
    china_map = load_china_map()
    
    # 1. 总体分布图
    print("\n生成总体分布图...")
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # 绘制地图底图
    plot_china_basemap(ax, china_map)
    
    # 绘制物种分布点
    for species in top_species:
        species_data = df_plot[df_plot['primary_species'] == species]
        ax.scatter(species_data['longitude'], species_data['latitude'],
                  c=[species_colors[species]], label=species, 
                  alpha=0.6, s=50, edgecolors='white', linewidth=0.5)
    
    ax.set_xlabel('经度', fontsize=14)
    ax.set_ylabel('纬度', fontsize=14)
    # ax.set_title('入侵物种空间分布图', fontsize=18, fontweight='bold', pad=20)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
    ax.grid(True, alpha=0.2, linestyle='--')
    ax.set_xlim(73, 136)
    ax.set_ylim(18, 54)
    
    plt.tight_layout()
    plt.savefig(output_dir / 'overall_distribution.png', dpi=300, bbox_inches='tight')
    print(f"✓ 已保存: overall_distribution.png")
    plt.close()
    
    # # 2. 按年份分布图
    # print("\n生成按年份分布图...")
    years = sorted(df_plot['year'].dropna().unique())
    
    # if len(years) > 0:
    #     # 选择关键年份绘制
    #     year_step = max(1, len(years) // 6)
    #     key_years = years[::year_step]
        
    #     n_cols = 3
    #     n_rows = (len(key_years) + n_cols - 1) // n_cols
        
    #     fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 5*n_rows))
    #     axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
        
    #     for idx, year in enumerate(key_years):
    #         if idx >= len(axes):
    #             break
            
    #         ax = axes[idx]
            
    #         # 绘制地图底图
    #         plot_china_basemap(ax, china_map)
            
    #         year_data = df_plot[df_plot['year'] == year]
            
    #         for species in top_species:
    #             species_data = year_data[year_data['primary_species'] == species]
    #             if len(species_data) > 0:
    #                 ax.scatter(species_data['longitude'], species_data['latitude'],
    #                          c=[species_colors[species]], label=species, 
    #                          alpha=0.6, s=40, edgecolors='white', linewidth=0.5)
            
    #         ax.set_title(f'{year}年 (n={len(year_data)})', fontsize=14, fontweight='bold')
    #         ax.set_xlabel('经度', fontsize=10)
    #         ax.set_ylabel('纬度', fontsize=10)
    #         ax.grid(True, alpha=0.2, linestyle='--')
    #         ax.set_xlim(73, 136)
    #         ax.set_ylim(18, 54)
        
    #     # 隐藏多余的子图
    #     for idx in range(len(key_years), len(axes)):
    #         axes[idx].set_visible(False)
        
    #     # 添加图例
    #     handles, labels = axes[0].get_legend_handles_labels()
    #     if handles:
    #         fig.legend(handles, labels, loc='center', bbox_to_anchor=(0.5, -0.02), 
    #                   ncol=5, fontsize=10)
        
    #     plt.tight_layout()
    #     plt.savefig(output_dir / 'distribution_by_year.png', dpi=300, bbox_inches='tight')
    #     print(f"✓ 已保存: distribution_by_year.png")
    #     plt.close()
    
    # 3. 时间序列累积图
    print("\n生成时间序列累积图...")
    fig, ax = plt.subplots(figsize=(16, 6))
    
    time_species_counts = []
    for year in sorted(years):
        year_data = df_plot[df_plot['year'] <= year]
        species_dist = year_data['primary_species'].value_counts()
        time_species_counts.append({'year': year, **species_dist.to_dict()})
    
    time_df = pd.DataFrame(time_species_counts).fillna(0)
    
    # 绘制堆叠面积图
    time_df_sorted = time_df.set_index('year')
    bottom = np.zeros(len(time_df_sorted))
    
    for species in top_species:
        if species in time_df_sorted.columns:
            values = time_df_sorted[species].values
            ax.fill_between(time_df_sorted.index, bottom, bottom + values, 
                           label=species, alpha=0.7, color=species_colors[species])
            bottom += values
    
    ax.set_xlabel('年份', fontsize=14)
    ax.set_ylabel('累积记录数', fontsize=14)
    # ax.set_title('入侵物种累积分布趋势', fontsize=18, fontweight='bold', pad=20)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    # 旋转x轴标签并调整间隔
    years_list = sorted(time_df_sorted.index.astype(str))
    if len(years_list) > 20:
        # 如果年份太多，只显示部分标签
        step = len(years_list) // 15
        ax.set_xticks(range(0, len(years_list), step))
        ax.set_xticklabels([years_list[i] for i in range(0, len(years_list), step)], 
                          rotation=45, ha='right')
    else:
        ax.set_xticklabels(years_list, rotation=45, ha='right')
    
    plt.tight_layout()
    plt.savefig(output_dir / 'cumulative_trend.png', dpi=300, bbox_inches='tight')
    print(f"✓ 已保存: cumulative_trend.png")
    plt.close()
    
    # 4. 热力图 - 物种×年份
    print("\n生成物种-年份热力图...")
    pivot_data = df_plot.groupby(['year', 'primary_species']).size().unstack(fill_value=0)
    
    # 只保留Top物种
    pivot_cols = [col for col in pivot_data.columns if col in top_species]
    pivot_data = pivot_data[pivot_cols]
    
    fig, ax = plt.subplots(figsize=(16, 6))
    sns.heatmap(pivot_data.T, cmap='YlOrRd', annot=False, fmt='d', 
                cbar_kws={'label': '记录数'}, ax=ax, linewidths=0.5)
    ax.set_xlabel('年份', fontsize=14)
    ax.set_ylabel('物种', fontsize=14)
    # ax.set_title('入侵物种时间分布热力图', fontsize=18, fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.savefig(output_dir / 'species_year_heatmap.png', dpi=300, bbox_inches='tight')
    print(f"✓ 已保存: species_year_heatmap.png")
    plt.close()
    
    # 5. Top物种的地理分布对比
    print("\n生成Top物种地理分布对比图...")
    top_5_species = species_counts.head(5).index.tolist()
    
    fig, axes = plt.subplots(2, 3, figsize=(24, 13))
    axes = axes.flatten()
    
    for idx, species in enumerate(top_5_species):
        if idx >= len(axes):
            break
        
        ax = axes[idx]
        
        # 绘制地图底图
        plot_china_basemap(ax, china_map)
        
        species_data = df_plot[df_plot['primary_species'] == species]
        
        ax.scatter(species_data['longitude'], species_data['latitude'],
                  c=[species_colors[species]], alpha=0.6, s=50, 
                  edgecolors='white', linewidth=0.5)
        
        ax.set_title(f'{species}\n(n={len(species_data)})', 
                    fontsize=12, fontweight='bold')
        ax.set_xlabel('经度', fontsize=10)
        ax.set_ylabel('纬度', fontsize=10)
        ax.grid(True, alpha=0.2, linestyle='--')
        ax.set_xlim(73, 136)
        ax.set_ylim(18, 54)
    
    # 最后一个子图显示所有物种
    ax = axes[5]
    
    # 绘制地图底图
    plot_china_basemap(ax, china_map)
    
    for species in top_5_species:
        species_data = df_plot[df_plot['primary_species'] == species]
        ax.scatter(species_data['longitude'], species_data['latitude'],
                  c=[species_colors[species]], label=species, 
                  alpha=0.5, s=30, edgecolors='white', linewidth=0.3)
    
    ax.set_title('所有Top5物种分布', fontsize=12, fontweight='bold')
    ax.set_xlabel('经度', fontsize=10)
    ax.set_ylabel('纬度', fontsize=10)
    ax.legend(fontsize=8, loc='best')
    ax.grid(True, alpha=0.2, linestyle='--')
    ax.set_xlim(73, 136)
    ax.set_ylim(18, 54)
    
    plt.tight_layout()
    plt.savefig(output_dir / 'top5_species_comparison.png', dpi=300, bbox_inches='tight')
    print(f"✓ 已保存: top5_species_comparison.png")
    plt.close()
    
    # 6. 密度分布图 (额外添加)
    print("\n生成密度分布热力图...")
    fig, ax = plt.subplots(figsize=(16, 12))
    
    # 绘制地图底图
    plot_china_basemap(ax, china_map)
    
    # 绘制六边形密度图
    hb = ax.hexbin(df_plot['longitude'], df_plot['latitude'], 
                   gridsize=30, cmap='YlOrRd', alpha=0.7, 
                   mincnt=1, edgecolors='white', linewidths=0.2)
    
    cb = plt.colorbar(hb, ax=ax)
    cb.set_label('记录密度', fontsize=12)
    
    ax.set_xlabel('经度 (Longitude)', fontsize=14)
    ax.set_ylabel('纬度 (Latitude)', fontsize=14)
    ax.set_title('入侵物种密度分布图', fontsize=18, fontweight='bold', pad=20)
    ax.grid(True, alpha=0.2, linestyle='--')
    ax.set_xlim(73, 136)
    ax.set_ylim(18, 54)
    
    plt.tight_layout()
    plt.savefig(output_dir / 'density_distribution.png', dpi=300, bbox_inches='tight')
    print(f"✓ 已保存: density_distribution.png")
    plt.close()
    
    # 生成总结
    print("\n" + "="*60)
    print("可视化完成")
    print("="*60)
    print(f"总记录数: {len(df_plot)}")
    print(f"时间范围: {years[0]} - {years[-1]}")
    print(f"可视化物种数: {len(top_species)}")
    print(f"输出目录: {output_dir}")
    print(f"\n生成的图片:")
    print(f"  1. overall_distribution.png - 总体空间分布")
    print(f"  2. distribution_by_year.png - 按年份分布")
    print(f"  3. cumulative_trend.png - 累积趋势")
    print(f"  4. species_year_heatmap.png - 时间热力图")
    print(f"  5. top5_species_comparison.png - Top5物种对比")
    print(f"  6. density_distribution.png - 密度分布图 (新增)")
    print("="*60)
    
    return df_plot


# 主函数
if __name__ == "__main__":
    # 执行可视化
    df_plot = visualize_species_distribution(
        'output/backups/backup_final_04334_retry_final.xlsx'
    )

读取文件: output/backups/backup_final_04334_retry_final.xlsx

数据行数: 4334
有效坐标数: 4334

物种数量: 38
前10种物种:
  互花米草: 852
  野燕麦: 765
  紫茎泽兰: 597
  空心莲子草: 412
  豚草: 363
  加拿大一枝黄花: 350
  薇甘菊: 331
  三裂叶豚草: 270
  飞机草: 247
  假高粱: 165

图片将保存到: output\backups\visualizations
正在加载中国地图底图...
✓ 地图加载成功

生成总体分布图...
✓ 已保存: overall_distribution.png

生成时间序列累积图...
✓ 已保存: cumulative_trend.png

生成物种-年份热力图...
✓ 已保存: species_year_heatmap.png

生成Top物种地理分布对比图...
✓ 已保存: top5_species_comparison.png

生成密度分布热力图...
✓ 已保存: density_distribution.png

可视化完成
总记录数: 4334
时间范围: 1911 - 2023
可视化物种数: 15
输出目录: output\backups\visualizations

生成的图片:
  1. overall_distribution.png - 总体空间分布
  2. distribution_by_year.png - 按年份分布
  3. cumulative_trend.png - 累积趋势
  4. species_year_heatmap.png - 时间热力图
  5. top5_species_comparison.png - Top5物种对比
  6. density_distribution.png - 密度分布图 (新增)
